<a href="https://colab.research.google.com/github/Sanjdcool/01_FCC_Projects_Responsive_Web_Design/blob/master/NSE_Data_Download.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import pandas as pd
from datetime import datetime
from io import BytesIO

class NSE():
    def __init__(self, timeout=10):
        self.base_url = 'https://www.nseindia.com'
        self.session = requests.sessions.Session()
        self.session.headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36 Edg/97.0.1072.55",
            "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
            "accept-language": "en-US,en;q=0.9"
        }
        self.timeout = timeout
        self.session.get(self.base_url, timeout=timeout)
        #self.cookies = []
    '''
    def __getCookies(self, renew=False):
        if len(self.cookies) > 0 and renew == False:
            return self.cookies
        r = requests.get(self.base_url, timeout=self.timeout, headers=self.headers)
        self.cookies = dict(r.cookies)
        return self.__getCookies()
    '''
    def getHistoricalData(self, symbol, series, from_date, to_date):
        try:
            url = "/api/historical/cm/equity?symbol={0}&series=[%22{1}%22]&from={2}&to={3}&csv=true".format(symbol.replace('&', '%26'), series, from_date.strftime('%d-%m-%Y'), to_date.strftime('%d-%m-%Y'))
            r = self.session.get(self.base_url + url, timeout=self.timeout)
            df = pd.read_csv(BytesIO(r.content), sep=',', thousands=',')
            df = df.rename(columns={'Date ': 'date', 'series ': 'series', 'OPEN ': 'open', 'HIGH ': 'high', 'LOW ': 'low', 'PREV. CLOSE ': 'prev_close', 'ltp ': 'ltp', 'close ': 'close', '52W H ': 'hi_52_wk', '52W L ': 'lo_52_wk', 'VOLUME ': 'trdqty', 'VALUE ': 'trdval', 'No of trades ': 'trades'})
            df.date = pd.to_datetime(df.date).dt.strftime('%Y-%m-%d')
            return df
        except:
            return None

    def fetch_index_from_nse(self, index_symbol):
        df = []
        res = self.session.get(self.base_url + '/api/equity-stockIndices?index=' + index_symbol, timeout=10)
        if res.status_code == 200:
            res_json = res.json()
            if 'data' in res_json:
                df = pd.json_normalize(res_json['data'])
            else:
                print('Data not returned from NSE')
                print(res_json)
        else:
            print('HTTP Request Failed: ')
            print(res)

        return df

    def save_index_to_csv(self, index_symbol, csv_file_name=None, delimiter=',', index=True, header=True):
        if csv_file_name == None:
            csv_file_name = index_symbol + '.csv'

        df = self.fetch_index_from_nse(index_symbol)
        if len(df) > 0:
            df.to_csv(csv_file_name, sep=delimiter, index=index, header=header)
            return True

        return False


if __name__ == '__main__':
    from datetime import date
    nse = NSE()
    df = nse.getHistoricalData('SBIN', 'EQ', date(2023,1,1), date(2023,5,20))
    print(df)
    print(nse.fetch_index_from_nse('NIFTY MIDCAP 50'))

          date series    open    high     low  prev_close     ltp   close  \
0   2023-05-19     EQ  579.00  586.25  569.90      574.20  575.65  575.15   
1   2023-05-18     EQ  590.30  593.90  571.40      586.30  576.35  574.20   
2   2023-05-17     EQ  586.00  590.50  579.50      586.30  587.20  586.30   
3   2023-05-16     EQ  584.00  590.40  582.00      581.90  585.95  586.30   
4   2023-05-15     EQ  578.75  584.00  574.65      578.15  582.05  581.90   
..         ...    ...     ...     ...     ...         ...     ...     ...   
88  2023-01-06     EQ  607.40  607.45  597.00      605.10  601.00  600.65   
89  2023-01-05     EQ  607.00  611.60  601.95      605.20  605.75  605.10   
90  2023-01-04     EQ  611.85  614.95  604.00      612.40  605.60  605.20   
91  2023-01-03     EQ  614.50  616.60  610.00      612.20  612.95  612.40   
92  2023-01-02     EQ  614.95  619.20  611.20      613.70  612.60  612.20   

     vwap   hi_52_wk  lo_52_wk    trdqty        trdval  trades  
0   576.07